In [116]:
import numpy as np
import matplotlib as plt

# PHYS 541 Final Project: The Cracker Barrel Peg Game
## Name: Jacob Buchanan

If you've ever been to a Cracker Barrel, you've probably made an attempt at the peg game. It's pretty straightforward. You have a triangle with some pegs and one open space to start. You have to jump all the pegs and get as few as you can at the end. It's difficult to get a good solution (I always end up doing worse the more I try). I'd like to study this game a bit by writing a simulation of it.

The triangle nature of this game makes simulating it somewhat tricky. I'll use a matrix and some defined Python classes to run this.

First I'll define a "space" class. It can hold a peg object and keeps track of activity.

In [117]:
class Space:
    def __init__(self, peg=None):
        self.peg = peg
    
    def set_peg(self, peg):
        self.peg = peg
        
    def remove_peg(self):
        peg = self.peg
        self.peg = None
        return peg

    def has_peg(self):
        if self.peg is not None:
            return True
        else:
            return False

I'll now define a peg. It knows how many moves it has made and if it's still in the game.

In [118]:
class Peg:
    def __init__(self, x, y):
        self.num_moves = 0
        self.in_game = True
        self.x = x
        self.y = y
        
    def move(self, to_x, to_y):
        self.x = to_x
        self.y = to_y
        self.num_moves += 1
    
    def remove(self):
        self.in_game = False

In [119]:
class Move:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
    
    def __str__(self):
        return f"Move peg at {self.p1} to {self.p2} which jumps over {self.jump_coord()}"
    
    def jump_coord(self):
        """
        gets the coordinate that the move jumps over
        :return: (x,y)
        """
        diff_x = self.p2[0] - self.p1[0]
        if diff_x > 0:
            jumped_x = self.p2[0] - 1
        elif diff_x < 0:
            jumped_x = self.p2[0] + 1
        else:
            jumped_x = self.p2[0]
        diff_y = self.p2[1] - self.p1[1]
        if diff_y > 0:
            jumped_y = self.p2[1] - 1
        elif diff_y < 0:
            jumped_y = self.p2[1] + 1
        else:
            jumped_y = self.p2[1]
        return jumped_x, jumped_y

Now I'll define the board. This is effectively a matrix with 5 rows and n columns (where n is the row number).

In [121]:
class Board:
    def __init__(self, empty_x=0, empty_y=0):
        self.board = [
            [Space()],
            [Space(), Space()],
            [Space(), Space(), Space()],
            [Space(), Space(), Space(), Space()],
            [Space(), Space(), Space(), Space(), Space()],
        ]
        self.setup_board((empty_x,empty_y))
        self.moves = []

    def setup_board(self, empty_pos=(0,0)):
        empty_space = False
        for i in range(len(self.board)):
            for j in range(len(self.board[i])):
                if not (i,j) == empty_pos:
                    self.board[i][j].set_peg(Peg(i,j))
                else:
                    empty_space = True
        if not empty_space:
            raise ValueError("One empty space required")
        
    def check_space(self, p):
        if p in self:
            return self.board[p[0]][p[1]]
        else:
            return None
        
    def check_move(self, move):
        """
        Checks a move from p1 to p2 (assuming the jump spacing is valid)
        :param p1: tuple (x,y)
        :param p2: tuple (x,y)
        :return: Boolean
        """
        if (move.p1 in self) and (move.p2 in self):
            if self.check_space(move.p1).has_peg():
                if not self.check_space(move.p2).has_peg():
                    if self.check_space(move.jump_coord()).has_peg():
                        return True
        return False
        
    def get_moves(self):
        moves = []
        for i in range(len(self.board)):
            for j in range(len(self.board[i])):
                if self.board[i][j].has_peg():
                    # It has a peg, can it move?
                    for dx in (-2, 0, 2):
                        for dy in (-2, 0, 2):
                            move = Move((i,j),(i+dx,j+dy))
                            if (dx, dy) == (0, 0):
                                continue
                            elif (i+dx < 0) or (j+dy < 0):
                                continue
                            elif dx*dy < 0:
                                # Avoid case where the peg jumps too far
                                continue
                            elif not self.check_move(move):
                                continue
                            else:
                                moves.append(move)
        self.moves = moves
        return moves
    
    def execute_move(self, move):
        if self.check_move(move):
            # Remove peg at jump coord
            jump_coord = move.jump_coord()
            self.check_space(jump_coord).remove_peg()
            # Move peg
            peg = self.check_space(move.p1).remove_peg()
            self.check_space(move.p2).set_peg(peg)
                    
    def __str__(self):
        output = ""
        for row in self.board:
            for space in row:
                if space.has_peg():
                    output += "1 "
                else:
                    output += "0 "
            output += "\n"
        return output
    
    def __contains__(self, item):
        """
        Check if coordinate is within the bounds of the board
        :param item: A tuple (x,y)
        :return: True or False
        """
        try:
            p = self.board[item[0]][item[1]]
            return True
        except IndexError:
            return False

board = Board(2,0)
print(board)
moves = board.get_moves()
board.execute_move(moves[0])
print(board)

1 
1 1 
0 1 1 
1 1 1 1 
1 1 1 1 1 

0 
0 1 
1 1 1 
1 1 1 1 
1 1 1 1 1 
